# 概要
学習済みのモデルを使って、歌詞からアーティスト名の予測＆LIMEによる解釈を可視化します。

Google Colaboratoryで実行することを想定しており、このNotebookだけでデモが完結します。

# 準備

ライブラリインストール

In [1]:
%%capture
!pip install japanize-matplotlib
!pip install lime
!pip install janome

GitHubから学習済みモデルなどをダウンロード

In [ ]:
!git clone https://github.com/matsuda-tkm/artist-prediction-xai.git

ライブラリのインポート

In [ ]:
import os
import sys
import torch
import functools
from tqdm import tqdm
import japanize_matplotlib
from collections import OrderedDict
from IPython.display import display, HTML
from lime.lime_text import LimeTextExplainer

sys.path.append('/content/artist-prediction-xai')  # Google Colabの場合
from network import CharacterCNN, CharacterCNNClassifier, CharacterCNNEmbedding
from utils import *

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

# 保存先のパス
PATH = '/content/artist-prediction-xai'  # Google Colabの場合

`models`に格納されている学習済みアーティスト名の一覧を表示します。

In [ ]:
for i,name in enumerate(os.listdir(os.path.join(PATH, "models"))):
    print(f'{i:2} : {name}')

上のセルで表示したIDを使って、分類対象とするアーティストを `select_id` で指定します。

すべてのアーティストを対象にする場合は、`select_all = True`にします。

In [ ]:
select_all = True #@param {type:"boolean"}

select_artist = dict()
select_id = [0,1,2,3]  #@param
for i,name in enumerate(os.listdir(os.path.join(PATH, 'models'))):
    select_artist[name] = True if (i in select_id) or select_all else False

print(f'You selected {sum(list(select_artist.values()))} artists.')

モデルの読み込み

In [ ]:
print('Loading models...')

# Embeddingの読み込み
state_dict = torch.load(os.path.join(PATH, 'pretrain/model_fold1.pth'), map_location=device)
state_dict = OrderedDict(list(state_dict.items())[0:1])
embed = CharacterCNNEmbedding().to(device)
embed.load_state_dict(state_dict)

# Classifierの読み込み
classifier = dict()
for artist in tqdm(os.listdir(os.path.join(PATH, 'models'))):
    if select_artist[artist]:
        model_list = []
        for file in os.listdir(os.path.join(PATH, 'models', artist)):
            state_dict = torch.load(os.path.join(PATH, 'models', artist, file), map_location=device)
            clf = CharacterCNNClassifier(2).to(device)
            clf.load_state_dict(state_dict)
            model_list.append(clf)
        classifier[artist] = model_list

artists = list(classifier.keys())
explainer = LimeTextExplainer(class_names=artists)

# 推論と解釈

## 1ブロック分の歌詞
- `txt`に歌詞1ブロック分を入力してください。
- `figsize`でグラフのサイズを調整できます。
- `num_samples`で、LIMEにおいてサンプルする近傍点の個数を指定できます。値が大きいほどLIMEの計算に時間がかかります。
- LIME表示用の分かち書きは自動生成されますが、`wakachi_txt`(カスタム分かち書き)に自分で分かち書きした歌詞も入れることができます。`txt`に入力した歌詞を**半角スペース**で分かち書きしたものを入れてください。

In [ ]:
# 歌詞1ブロック分 #################
txt = """麦わらの帽子の君が
揺れたマリーゴールドに似てる
あれは空がまだ青い夏のこと
懐かしいと笑えたあの日の恋"""

figsize = (18,3)  # グラフのサイズを調整 (横,縦)
num_samples = 3  # サンプルする近傍点の個数

# 分類結果の取得
prob = predict_one_block(txt, embed, classifier, device)
# 棒グラフで可視化
show_predict_one_block(prob, artists, sort=True, figsize=figsize)

# カスタム分かち書き ###############
wakachi_txt = """
"""

# LIME
if wakachi_txt == '\n':
    wakachi_txt = wakachi_one_block(txt)
predict_some_block_lime = functools.partial(predict_some_block,  embed=embed, classifier=classifier, device=device)
exp = explainer.explain_instance(wakachi_txt, predict_some_block_lime, num_features=len(wakachi_txt.split()), labels=range(len(artists)), num_samples=num_samples)
highlighted_text = highlight(exp, wakachi_txt, artists, sort_by=prob)
display(HTML(highlighted_text))

## 歌詞全体
- `txt`に歌詞全体を入力してください。ブロック間は改行を1つはさんでください。
- `figsize`でグラフのサイズを調整できます。
- `num_samples`で、LIMEにおいてサンプルする近傍点の個数を指定できます。値が大きいほどLIMEの計算に時間がかかります。

In [ ]:
# 歌詞全体 #######################
txt = """
風の強さがちょっと
心を揺さぶりすぎて
真面目に見つめた
君が恋しい

でんぐり返しの日々
可哀想なふりをして
だらけてみたけど
希望の光は

目の前でずっと輝いている
幸せだ

麦わらの帽子の君が
揺れたマリーゴールドに似てる
あれは空がまだ青い夏のこと
懐かしいと笑えたあの日の恋

「もう離れないで」と
泣きそうな目で見つめる君を
雲のような優しさでそっとぎゅっと
抱きしめて　抱きしめて　離さない

本当の気持ち全部
吐き出せるほど強くはない
でも不思議なくらいに
絶望は見えない

目の奥にずっと写るシルエット
大好きさ

柔らかな肌を寄せあい
少し冷たい空気を2人
かみしめて歩く今日という日に
何と名前をつけようかなんて話して

ああ　アイラブユーの言葉じゃ
足りないからとキスして
雲がまだ2人の影を残すから
いつまでも　いつまでも　このまま

遥か遠い場所にいても
繋がっていたいなあ
2人の想いが
同じでありますように

麦わらの帽子の君が
揺れたマリーゴールドに似てる
あれは空がまだ青い夏のこと
懐かしいと笑えたあの日の恋

「もう離れないで」と
泣きそうな目で見つめる君を
雲のような優しさでそっとぎゅっと
抱きしめて離さない

ああ　アイラブユーの言葉じゃ
足りないからとキスして
雲がまだ2人の影を残すから
いつまでも　いつまでも　このまま

離さない
いつまでも　いつまでも　離さない
"""
figsize = (18,10)  # グラフのサイズを調整 (横,縦)
num_samples = 3  # サンプルする近傍点の個数

# 分類結果の取得
prob = predict_some_block(txt.split('\n\n'), embed, classifier, device)

# 100%積み上げ棒グラフで可視化
show_predict_whole_song(prob, artists, sort=True, raw_txt_arr=txt.split('\n\n'), figsize=figsize)

# LIME
wakachi_txt = wakachi_some_block(txt)
predict_whole_song_lime = functools.partial(predict_whole_song, embed=embed, classifier=classifier, device=device)
exp = explainer.explain_instance(wakachi_txt, predict_whole_song_lime, num_features=len(wakachi_txt.split()), labels=range(len(artists)), num_samples=num_samples)
highlighted_text = highlight(exp, wakachi_txt, artists, sort_by=prob.mean(axis=0))
display(HTML(highlighted_text))